In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
df1 = pd.read_csv("data/2024.csv", sep=";")
df1

,hub_id,Days in full_date,Average bike_count,# of unique dropoff_hub_id,# of unique pickup_hub_id
0,2180.0,01/01/2024,6.282412,NaN,NaN
1,2180.0,02/01/2024,6.080100,NaN,NaN
2,2180.0,04/01/2024,6.078173,NaN,NaN
3,2180.0,05/01/2024,5.936238,NaN,NaN
4,2180.0,07/01/2024,6.443539,NaN,NaN
...,...,...,...,...,...
6925,26889.0,14/09/2024,6.150918,21.0,20.0
6926,26889.0,15/09/2024,6.229018,12.0,12.0
6927,26889.0,16/09/2024,6.177963,11.0,14.0
6928,26889.0,17/09/2024,6.418395,16.0,18.0


In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6930 entries, 0 to 6929
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   hub_id                      6930 non-null   float64
 1   Days in full_date           6930 non-null   object 
 2   Average bike_count          6930 non-null   float64
 3   # of unique dropoff_hub_id  6707 non-null   float64
 4   # of unique pickup_hub_id   6707 non-null   float64
dtypes: float64(4), object(1)
memory usage: 270.8+ KB


In [22]:
df2 = pd.read_csv("data/alltime.csv")
df2.id.unique()

array([30467, 30466, 30464, 30434, 30420, 30390, 30282, 30207, 30185,
       30156, 30155, 30151, 30147, 30146, 30145, 30126, 30121, 30120,
       30095, 30084, 30027, 30026, 30025, 30024, 30023, 30022, 30011,
       29982, 29974, 29965, 29964, 29963, 29946, 29942, 29941, 29940,
       29939, 29938, 29937, 29936, 29935, 29934, 29933, 29745, 29663,
       29347, 29346, 29345, 29253, 29224, 29200, 29198, 29147, 29112,
       29109, 29083, 29049, 28932, 28814, 28813, 28812, 28811, 28804,
       28803, 28802, 28801, 28800, 28799, 28798, 28797, 28796, 28795,
       28794, 28793, 28792, 28791, 28790, 28789, 28749, 28738, 28264,
       27717, 27567, 27452, 27451, 27450, 27449, 27448, 27447, 27446,
       27445, 27444, 27443, 27442, 27441, 27440, 27382, 27381, 27380,
       27379, 27378, 27377, 27376, 27375, 27374, 27373, 27372, 27209,
       27183, 26889, 26805, 26730, 26725, 26724, 26708, 26653, 26224,
       26209, 24602, 24482, 24481, 24479, 24478, 24477, 24476, 24475,
       24474, 24473,

In [23]:
df3 = pd.merge(left=df1, right=df2, how="inner", left_on="hub_id", right_on="id")

#columns_to_drop = ["id", "state", "created_at", "pickups", "dropoffs"]
df3 = df3.drop(columns=["id", "state", "created_at", "pickups", "dropoffs"])

df3.hub_id.unique()

array([24367., 24368., 24369., 24370., 24371., 24373., 24375., 24376.,
       24379., 24380., 24381., 24383., 24385., 24389., 24391., 24392.,
       24393., 24395., 24397., 24399., 24408., 24412., 24413., 24415.,
       24417., 24419., 24436., 24457., 24465., 24466., 24467., 24469.,
       24471., 24481., 26224., 26889.])

In [6]:
# Create a new column that is the difference between '# of unique dropoff_hub_id' and '# of unique pickup_hub_id'
df3['dropoff_pickup_diff'] = df3['# of unique dropoff_hub_id'] - df3['# of unique pickup_hub_id']

# Display the first few rows to verify the result
print(df3[['# of unique dropoff_hub_id', '# of unique pickup_hub_id', 'dropoff_pickup_diff']].head())


   # of unique dropoff_hub_id  # of unique pickup_hub_id  dropoff_pickup_diff
0                        10.0                       10.0                  0.0
1                         2.0                        2.0                  0.0
2                         7.0                        6.0                  1.0
3                         5.0                        4.0                  1.0
4                         4.0                        4.0                  0.0


In [7]:
# Create a new column that is the result of 'Average bike_count' minus 'dropoff_pickup_diff'
df3['bike_count_adjusted'] = df3['Average bike_count'] - df3['dropoff_pickup_diff']

# Display the first few rows to verify the result
print(df3[['Average bike_count', 'dropoff_pickup_diff', 'bike_count_adjusted']].head())


   Average bike_count  dropoff_pickup_diff  bike_count_adjusted
0            6.282412                  0.0             6.282412
1            6.080100                  0.0             6.080100
2            5.396247                  1.0             4.396247
3            6.078173                  1.0             5.078173
4            5.936238                  0.0             5.936238


In [9]:
# First, make sure the dataframe is sorted by 'name' and 'Days in full_date'
#df3 = df3.sort_values(by=['name', 'Days in full_date'])

# Define a function to check if 'Average bike_count' has been continuously decreasing over the last 5 days
def check_decreasing(group):
    # Create a column to store the result
    decreasing_trend = [False] * len(group)  # Default value is False
    
    # Iterate through the group, starting from the 5th row
    for i in range(5, len(group)):
        # Get the 'Average bike_count' values for the current row and the previous 5 days
        last_5_days = group['Average bike_count'].iloc[i-5:i]
        
        # Check if the values are strictly decreasing
        if all(x > y for x, y in zip(last_5_days, last_5_days[1:])):
            decreasing_trend[i] = True  # Set to True if all values are decreasing
    
    # Add the result as a new column in the group
    group['decreasing_trend'] = decreasing_trend
    return group

# Apply the function to each group of 'name'
df3 = df3.groupby('name').apply(check_decreasing)

df3.decreasing_trend



/var/folders/jz/kmgcfbj10t539vkl1kk0smzm0000gn/T/ipykernel_4668/925174170.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df3 = df3.groupby('name').apply(check_decreasing)


name                               
 Wik, Kanalfähre - Busstation  6193    False
                               6194    False
                               6195    False
                               6196    False
                               6197    False
                                       ...  
thyssenkrupp Marine Systems    4221    False
                               4222    False
                               4223    False
                               4224    False
                               4225    False
Name: decreasing_trend, Length: 6707, dtype: bool

In [10]:
df3.info()


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6707 entries, (' Wik, Kanalfähre - Busstation', np.int64(6193)) to ('thyssenkrupp Marine Systems', np.int64(4225))
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   hub_id                      6707 non-null   float64
 1   Days in full_date           6707 non-null   object 
 2   Average bike_count          6707 non-null   float64
 3   # of unique dropoff_hub_id  6707 non-null   float64
 4   # of unique pickup_hub_id   6707 non-null   float64
 5   name                        6707 non-null   object 
 6   maximum_capacity            6707 non-null   int64  
 7   dropoff_pickup_diff         6707 non-null   float64
 8   bike_count_adjusted         6707 non-null   float64
 9   decreasing_trend            6707 non-null   bool   
dtypes: bool(1), float64(6), int64(1), object(2)
memory usage: 809.6+ KB


In [11]:
df3.decreasing_trend.value_counts()

decreasing_trend
False    6637
True       70
Name: count, dtype: int64

In [12]:
df3.reset_index(drop=True)

,hub_id,Days in full_date,Average bike_count,# of unique dropoff_hub_id,# of unique pickup_hub_id,name,maximum_capacity,dropoff_pickup_diff,bike_count_adjusted,decreasing_trend
0,26224.0,01/01/2024,6.282412,9.0,9.0,"Wik, Kanalfähre - Busstation",20,0.0,6.282412,False
1,26224.0,02/01/2024,6.080100,4.0,5.0,"Wik, Kanalfähre - Busstation",20,-1.0,7.080100,False
2,26224.0,03/01/2024,5.396247,4.0,4.0,"Wik, Kanalfähre - Busstation",20,0.0,5.396247,False
3,26224.0,04/01/2024,6.078173,9.0,10.0,"Wik, Kanalfähre - Busstation",20,-1.0,7.078173,False
4,26224.0,05/01/2024,5.936238,6.0,6.0,"Wik, Kanalfähre - Busstation",20,0.0,5.936238,False
...,...,...,...,...,...,...,...,...,...,...
6702,24408.0,14/09/2024,6.150918,4.0,5.0,thyssenkrupp Marine Systems,25,-1.0,7.150918,False
6703,24408.0,15/09/2024,6.229018,13.0,12.0,thyssenkrupp Marine Systems,25,1.0,5.229018,False
6704,24408.0,16/09/2024,6.177963,22.0,27.0,thyssenkrupp Marine Systems,25,-5.0,11.177963,False
6705,24408.0,17/09/2024,6.418395,11.0,16.0,thyssenkrupp Marine Systems,25,-5.0,11.418395,False


In [14]:
# Check the current index names
print("Current index names:", df3.index.names)

# Drop a specific index level (e.g., 'name')
df3 = df3.reset_index(level='name', drop=True)

# Check the index names again to confirm
print("Index names after dropping:", df3.index.names)



Current index names: ['name', None]
Index names after dropping: [None]


In [16]:
# Check if 'name' is in the index, and reset index if it is
#if 'name' in df3.index.names:
#    df3 = df3.reset_index()

# Now, ensure 'name' is a regular column, not part of the index
#df3 = df3.reset_index(drop=True)

# Convert 'Days in full_date' to datetime, assuming the format is day-first (DD/MM/YYYY)
df3['Days in full_date'] = pd.to_datetime(df3['Days in full_date'], dayfirst=True, errors='coerce')

# Now continue with your operations, such as extracting 'Year-Month'
df3['Year-Month'] = df3['Days in full_date'].dt.to_period('M')



# Perform the groupby operation
df_agg = df3.groupby(['name', 'Year-Month']).agg({
    'Average bike_count': ['mean', 'min', 'max'],
    '# of unique dropoff_hub_id': ['mean', 'min', 'max'],
    '# of unique pickup_hub_id': ['mean', 'min', 'max'],
    'dropoff_pickup_diff': ['mean', 'min', 'max'],
    'decreasing_trend': lambda x: (x == True).sum(),  # Count True instances
    'maximum_capacity': 'first'
})

# Display the aggregated dataframe
df_agg



Average bike_count            \
                                                       mean       min   
name                          Year-Month                                
 Wik, Kanalfähre - Busstation 2024-01              7.281682  5.396247   
                              2024-02              8.257237  6.096234   
                              2024-03              6.131662  4.012262   
                              2024-04              5.773726  4.152251   
                              2024-05              5.184003  4.110842   
...                                                     ...       ...   
thyssenkrupp Marine Systems   2024-05              5.181806  4.110842   
                              2024-06              5.447435  4.095550   
                              2024-07              7.140228  4.474172   
                              2024-08              7.880128  5.657152   
                              2024-09              6.737242  4.847843   

                                                     \
                                                max   
name                          Year-Month              
 Wik, Kanalfähre - Busstation 2024-01      9.255474   
                              2024-02     13.067517   
                              2024-03      9.061312   
                              2024-04      7.113191   
                              2024-05      7.093038   
...                                             ...   
thyssenkrupp Marine Systems   2024-05      7.093038   
                              2024-06      7.473222   
                              2024-07      8.928098   
                              2024-08     11.000826   
                              2024-09      8.595353   

                                         # of unique dropoff_hub_id       \
                                                               mean  min   
name                          Year-Month                                   
 Wik, Kanalfähre - Busstation 2024-01                     10.419355  3.0   
                              2024-02                     12.172414  1.0   
                              2024-03                     15.933333  3.0   
                              2024-04                     12.344828  7.0   
                              2024-05                     15.483871  3.0   
...                                                             ...  ...   
thyssenkrupp Marine Systems   2024-05                      9.066667  1.0   
                              2024-06                     12.482759  1.0   
                              2024-07                     13.000000  1.0   
                              2024-08                     10.258065  2.0   
                              2024-09                      9.944444  2.0   

                                               # of unique pickup_hub_id       \
                                           max                      mean  min   
name                          Year-Month                                        
 Wik, Kanalfähre - Busstation 2024-01     19.0                 10.548387  4.0   
                              2024-02     26.0                 12.379310  1.0   
                              2024-03     29.0                 16.133333  3.0   
                              2024-04     25.0                 12.620690  6.0   
                              2024-05     26.0                 15.548387  3.0   
...                                        ...                       ...  ...   
thyssenkrupp Marine Systems   2024-05     22.0                  9.500000  1.0   
                              2024-06     22.0                 12.586207  1.0   
                              2024-07     21.0                 13.193548  1.0   
                              2024-08     20.0                 10.225806  2.0   
                              2024-09     22.0                 10.333333  2.0   

                                               

In [17]:
len(df3.name.unique())

36

In [21]:
import pandas as pd
import plotly.graph_objects as go

# Convert 'Days in full_date' to datetime, assuming the format is day-first (DD/MM/YYYY)
df3['Days in full_date'] = pd.to_datetime(df3['Days in full_date'], dayfirst=True, errors='coerce')

# Extract 'Year-Month'
df3['Year-Month'] = df3['Days in full_date'].dt.to_period('M')

# Group by 'name' and 'Year-Month' to calculate average, min, and max for the values
df_agg = df3.groupby(['name', 'Year-Month']).agg({
    'Average bike_count': ['mean', 'min', 'max'],
    '# of unique dropoff_hub_id': ['mean', 'min', 'max'],
    '# of unique pickup_hub_id': ['mean', 'min', 'max'],
    'dropoff_pickup_diff': ['mean', 'min', 'max'],
    'decreasing_trend': lambda x: (x == True).sum(),  # Count True instances
    'maximum_capacity': 'first'
}).reset_index()

# Flatten MultiIndex columns
df_agg.columns = ['name', 'Year-Month', 
                  'avg_bike_count', 'min_bike_count', 'max_bike_count',
                  'avg_dropoff', 'min_dropoff', 'max_dropoff',
                  'avg_pickup', 'min_pickup', 'max_pickup',
                  'dropoff_pickup_diff_mean', 'dropoff_pickup_diff_min', 'dropoff_pickup_diff_max',
                  'decreasing_trend_count', 'maximum_capacity']

# Initialize the plotly figure
fig = go.Figure()

# Create traces for each 'name' value with a slider
names = df_agg['name'].unique()

# Loop over each 'name' and create corresponding traces
for name in names:
    df_filtered = df_agg[df_agg['name'] == name]
    
    # Add average line
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), 
                             y=df_filtered['avg_bike_count'], 
                             mode='lines+markers', name=f'{name} - Avg Bike Count', visible=False))
    
    # Add max line
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), 
                             y=df_filtered['max_bike_count'], 
                             mode='lines', name=f'{name} - Max Bike Count', visible=False, line=dict(dash='dash')))
    
    # Add min line
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), 
                             y=df_filtered['min_bike_count'], 
                             mode='lines', name=f'{name} - Min Bike Count', visible=False, line=dict(dash='dot')))
    
    # Add decreasing trend count as a bar trace
    fig.add_trace(go.Bar(x=df_filtered['Year-Month'].astype(str), 
                         y=df_filtered['decreasing_trend_count'], 
                         name=f'{name} - Decreasing Trend Count', 
                         visible=False, 
                         marker_color='rgba(255, 99, 132, 0.7)'))  # Adjust color as needed
    
    # Add maximum capacity line
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), 
                             y=[df_filtered['maximum_capacity'].values[0]] * len(df_filtered), 
                             mode='lines', 
                             name=f'{name} - Max Capacity', 
                             line=dict(color='red', dash='dash'),
                             visible=False))

# Make the first 'name' visible by default
for i in range(5):  # The first 5 traces (avg, max, min, decreasing trend count, max capacity) of the first 'name'
    fig.data[i].visible = True

# Create steps for the slider (one step for each 'name')
steps = []
for i, name in enumerate(names):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},  # Hide all traces initially
              {"title": f"Bike Data for {name}"}],  # Update the plot title based on the 'name'
    )
    
    # Make the corresponding traces for this 'name' visible
    step["args"][0]["visible"][i*5] = True  # Show average line
    step["args"][0]["visible"][i*5 + 1] = True  # Show max line
    step["args"][0]["visible"][i*5 + 2] = True  # Show min line
    step["args"][0]["visible"][i*5 + 3] = True  # Show decreasing trend count
    step["args"][0]["visible"][i*5 + 4] = True  # Show max capacity line
    steps.append(step)

# Create slider
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Name: "},
    pad={"t": 50},
    steps=steps
)]

# Update layout
fig.update_layout(
    sliders=sliders,
    title="Bike Metrics (Average, Max, Min, Decreasing Trend Count, Max Capacity) Over Time",
    xaxis_title="Month",
    yaxis_title="Values",
    yaxis_range=[0, 40],  # Set the y-axis range from 0 to 60
    legend_title="Metrics"
)

# Show plot
fig.show()


What would be the optimum here?

Average in the middle of the station capacity with 

In [30]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Convert 'Days in full_date' to datetime, assuming the format is day-first (DD/MM/YYYY)
df3['Days in full_date'] = pd.to_datetime(df3['Days in full_date'], dayfirst=True, errors='coerce')

# Now continue with your operations, such as extracting 'Year-Month'
df3['Year-Month'] = df3['Days in full_date'].dt.to_period('M')


# Group by 'name' and 'Year-Month' to calculate average, min, and max for the values
df_agg = df3.groupby(['name', 'Year-Month']).agg({
    'Average bike_count': ['mean', 'min', 'max'],
    '# of unique dropoff_hub_id': ['mean', 'min', 'max'],
    '# of unique pickup_hub_id': ['mean', 'min', 'max']
}).reset_index()

# Flatten MultiIndex columns
df_agg.columns = ['name', 'Year-Month', 'avg_bike_count', 'min_bike_count', 'max_bike_count',
                  'avg_dropoff', 'min_dropoff', 'max_dropoff',
                  'avg_pickup', 'min_pickup', 'max_pickup']

# Initialize the plotly figure
fig = go.Figure()

# Create traces for each 'name' value with a slider
names = df_agg['name'].unique()

# Loop over each 'name' and create corresponding traces for avg, min, max values
for name in names:
    df_filtered = df_agg[df_agg['name'] == name]
    
    # Add average line
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['avg_bike_count'], 
                             mode='lines+markers', name=f'{name} - Avg Bike Count', visible=False))
    
    # Add max line
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['max_bike_count'], 
                             mode='lines', name=f'{name} - Max Bike Count', visible=False, line=dict(dash='dash')))
    
    # Add min line
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['min_bike_count'], 
                             mode='lines', name=f'{name} - Min Bike Count', visible=False, line=dict(dash='dot')))

# Make the first 'name' visible by default
for i in range(3):  # The first 3 traces (avg, max, min) of the first 'name'
    fig.data[i].visible = True

# Create steps for the slider (one step for each 'name')
steps = []
for i, name in enumerate(names):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},  # Hide all traces initially
              {"title": f"Bike Data for {name}"}],  # Update the plot title based on the 'name'
    )
    
    # Make the corresponding traces for this 'name' visible
    step["args"][0]["visible"][i*3] = True  # Show average line
    step["args"][0]["visible"][i*3 + 1] = True  # Show max line
    step["args"][0]["visible"][i*3 + 2] = True  # Show min line
    steps.append(step)

# Create slider
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Name: "},
    pad={"t": 50},
    steps=steps
)]

# Update layout
fig.update_layout(
    sliders=sliders,
    title="Bike Metrics (Average, Max, Min) Over Time",
    xaxis_title="Month",
    yaxis_title="Values",
    legend_title="Metrics"
)

# Show plot
fig.show()


In [31]:
import pandas as pd
import plotly.graph_objects as go

# Assuming 'Days in full_date' is already in datetime format
df3['Days in full_date'] = pd.to_datetime(df3['Days in full_date'], dayfirst=True)

# Create a new column for Year-Month
df3['Year-Month'] = df3['Days in full_date'].dt.to_period('M')

# Group by 'name' and 'Year-Month' to calculate average, min, and max for the values
df_agg = df3.groupby(['name', 'Year-Month']).agg({
    'Average bike_count': ['mean', 'min', 'max'],
    '# of unique dropoff_hub_id': ['mean', 'min', 'max'],
    '# of unique pickup_hub_id': ['mean', 'min', 'max'],
    'maximum_capacity': 'first'  # Since maximum_capacity is the same for each 'name', just take the first
}).reset_index()

# Flatten MultiIndex columns
df_agg.columns = ['name', 'Year-Month', 'avg_bike_count', 'min_bike_count', 'max_bike_count',
                  'avg_dropoff', 'min_dropoff', 'max_dropoff',
                  'avg_pickup', 'min_pickup', 'max_pickup', 'maximum_capacity']

# Initialize the plotly figure
fig = go.Figure()

# Create traces for each 'name' value with a slider
names = df_agg['name'].unique()

# Loop over each 'name' and create corresponding traces for avg, min, max values and maximum_capacity
for name in names:
    df_filtered = df_agg[df_agg['name'] == name]
    
    # Add average line
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['avg_bike_count'], 
                             mode='lines+markers', name=f'{name} - Avg Bike Count', visible=False))
    
    # Add max line
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['max_bike_count'], 
                             mode='lines', name=f'{name} - Max Bike Count', visible=False, line=dict(dash='dash')))
    
    # Add min line
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['min_bike_count'], 
                             mode='lines', name=f'{name} - Min Bike Count', visible=False, line=dict(dash='dot')))
    
    # Add horizontal dashed red line for maximum_capacity
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), 
                             y=[df_filtered['maximum_capacity'].values[0]] * len(df_filtered),
                             mode='lines', name=f'{name} - Max Capacity', visible=False,
                             line=dict(color='red', dash='dash'), showlegend=False))

# Make the first 'name' visible by default
for i in range(4):  # The first 4 traces (avg, max, min, and max_capacity) of the first 'name'
    fig.data[i].visible = True

# Create steps for the slider (one step for each 'name')
steps = []
for i, name in enumerate(names):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},  # Hide all traces initially
              {"title": f"Bike Data for {name}"}],  # Update the plot title based on the 'name'
    )
    
    # Make the corresponding traces for this 'name' visible
    step["args"][0]["visible"][i*4] = True  # Show average line
    step["args"][0]["visible"][i*4 + 1] = True  # Show max line
    step["args"][0]["visible"][i*4 + 2] = True  # Show min line
    step["args"][0]["visible"][i*4 + 3] = True  # Show max_capacity line
    steps.append(step)

# Create slider
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Name: "},
    pad={"t": 50},
    steps=steps
)]

# Update layout
fig.update_layout(
    sliders=sliders,
    title="Bike Metrics (Average, Max, Min, Max Capacity) Over Time",
    xaxis_title="Month",
    yaxis_title="Values",
    legend_title="Metrics"
)

# Show plot
fig.show()


In [32]:
import pandas as pd
import plotly.graph_objects as go

# Assuming 'Days in full_date' is already in datetime format
df3['Days in full_date'] = pd.to_datetime(df3['Days in full_date'], dayfirst=True)

# Create a new column for Year-Month
df3['Year-Month'] = df3['Days in full_date'].dt.to_period('M')

# Group by 'name' and 'Year-Month' to calculate average, min, and max for the values
df_agg = df3.groupby(['name', 'Year-Month']).agg({
    'Average bike_count': ['mean', 'min', 'max'],
    '# of unique dropoff_hub_id': ['mean', 'min', 'max'],
    '# of unique pickup_hub_id': ['mean', 'min', 'max'],
    'maximum_capacity': 'first'  # Since maximum_capacity is the same for each 'name', just take the first
}).reset_index()

# Flatten MultiIndex columns
df_agg.columns = ['name', 'Year-Month', 'avg_bike_count', 'min_bike_count', 'max_bike_count',
                  'avg_dropoff', 'min_dropoff', 'max_dropoff',
                  'avg_pickup', 'min_pickup', 'max_pickup', 'maximum_capacity']

# Initialize the plotly figure
fig = go.Figure()

# Create traces for each 'name' value with a slider
names = df_agg['name'].unique()

# Loop over each 'name' and create corresponding traces for avg, min, max values and maximum_capacity
for name in names:
    df_filtered = df_agg[df_agg['name'] == name]
    
    # Add average line
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['avg_bike_count'], 
                             mode='lines+markers', name=f'{name} - Avg Bike Count', visible=False))
    
    # Add max line
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['max_bike_count'], 
                             mode='lines', name=f'{name} - Max Bike Count', visible=False, line=dict(dash='dash')))
    
    # Add min line
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['min_bike_count'], 
                             mode='lines', name=f'{name} - Min Bike Count', visible=False, line=dict(dash='dot')))
    
    # Add horizontal dashed red line for maximum_capacity
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), 
                             y=[df_filtered['maximum_capacity'].values[0]] * len(df_filtered),
                             mode='lines', name=f'{name} - Max Capacity', visible=False,
                             line=dict(color='red', dash='dash'), showlegend=False))

# Make the first 'name' visible by default
for i in range(4):  # The first 4 traces (avg, max, min, and max_capacity) of the first 'name'
    fig.data[i].visible = True

# Create steps for the slider (one step for each 'name')
steps = []
for i, name in enumerate(names):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},  # Hide all traces initially
              {"title": f"Bike Data for {name}"}],  # Update the plot title based on the 'name'
    )
    
    # Make the corresponding traces for this 'name' visible
    step["args"][0]["visible"][i*4] = True  # Show average line
    step["args"][0]["visible"][i*4 + 1] = True  # Show max line
    step["args"][0]["visible"][i*4 + 2] = True  # Show min line
    step["args"][0]["visible"][i*4 + 3] = True  # Show max_capacity line
    steps.append(step)

# Create slider
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Name: "},
    pad={"t": 50},
    steps=steps
)]

# Update layout to include a fixed y-axis range (0 to 50)
fig.update_layout(
    sliders=sliders,
    title="Bike Metrics (Average, Max, Min, Max Capacity) Over Time",
    xaxis_title="Month",
    yaxis_title="Values",
    yaxis_range=[0, 50],  # Set y-axis range from 0 to 50
    legend_title="Metrics"
)

# Show plot
fig.show()


In [35]:
import pandas as pd
import plotly.graph_objects as go

# Assuming 'Days in full_date' is already in datetime format
df3['Days in full_date'] = pd.to_datetime(df3['Days in full_date'], dayfirst=True)

# Create a new column for Year-Month
df3['Year-Month'] = df3['Days in full_date'].dt.to_period('M')

# Create a new column for dropoff_pickup_diff
df3['dropoff_pickup_diff'] = df3['# of unique dropoff_hub_id'] - df3['# of unique pickup_hub_id']

# Group by 'name' and 'Year-Month' to calculate average, min, and max for the values
df_agg = df3.groupby(['name', 'Year-Month']).agg({
    'Average bike_count': ['mean', 'min', 'max'],
    '# of unique dropoff_hub_id': ['mean', 'min', 'max'],
    '# of unique pickup_hub_id': ['mean', 'min', 'max'],
    'dropoff_pickup_diff': ['mean', 'min', 'max'],  # Aggregation for the new column
    'maximum_capacity': 'first'  # Since maximum_capacity is the same for each 'name', just take the first
}).reset_index()

# Flatten MultiIndex columns
df_agg.columns = ['name', 'Year-Month', 'avg_bike_count', 'min_bike_count', 'max_bike_count',
                  'avg_dropoff', 'min_dropoff', 'max_dropoff',
                  'avg_pickup', 'min_pickup', 'max_pickup',
                  'avg_diff', 'min_diff', 'max_diff', 'maximum_capacity']

# Initialize the plotly figure
fig = go.Figure()

# Create traces for each 'name' value with a slider
names = df_agg['name'].unique()

# Loop over each 'name' and create corresponding traces for avg, min, max values and maximum_capacity
for name in names:
    df_filtered = df_agg[df_agg['name'] == name]
    
    # Add average line for bike count
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['avg_bike_count'], 
                             mode='lines+markers', name=f'{name} - Avg Bike Count', visible=False))
    
    # Add max line for bike count
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['max_bike_count'], 
                             mode='lines', name=f'{name} - Max Bike Count', visible=False, line=dict(dash='dash')))
    
    # Add min line for bike count
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['min_bike_count'], 
                             mode='lines', name=f'{name} - Min Bike Count', visible=False, line=dict(dash='dot')))
    
    # Add horizontal dashed red line for maximum_capacity
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), 
                             y=[df_filtered['maximum_capacity'].values[0]] * len(df_filtered),
                             mode='lines', name=f'{name} - Max Capacity', visible=False,
                             line=dict(color='red', dash='dash'), showlegend=False))

    # Add average line for dropoff_pickup_diff
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['avg_diff'], 
                             mode='lines+markers', name=f'{name} - Avg Dropoff-Pickup Diff', visible=False))
    
    # Add max line for dropoff_pickup_diff
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['max_diff'], 
                             mode='lines', name=f'{name} - Max Dropoff-Pickup Diff', visible=False, line=dict(dash='dash')))
    
    # Add min line for dropoff_pickup_diff
    fig.add_trace(go.Scatter(x=df_filtered['Year-Month'].astype(str), y=df_filtered['min_diff'], 
                             mode='lines', name=f'{name} - Min Dropoff-Pickup Diff', visible=False, line=dict(dash='dot')))

# Make the first 'name' visible by default
for i in range(7):  # The first 7 traces (avg, max, min, and max_capacity for bike count, dropoff-pickup diff) of the first 'name'
    fig.data[i].visible = True

# Create steps for the slider (one step for each 'name')
steps = []
for i, name in enumerate(names):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},  # Hide all traces initially
              {"title": f"Bike Data for {name}"}],  # Update the plot title based on the 'name'
    )
    
    # Make the corresponding traces for this 'name' visible
    step["args"][0]["visible"][i*7] = True  # Show avg bike count
    step["args"][0]["visible"][i*7 + 1] = True  # Show max bike count
    step["args"][0]["visible"][i*7 + 2] = True  # Show min bike count
    step["args"][0]["visible"][i*7 + 3] = True  # Show max capacity
    step["args"][0]["visible"][i*7 + 4] = True  # Show avg dropoff-pickup diff
    step["args"][0]["visible"][i*7 + 5] = True  # Show max dropoff-pickup diff
    step["args"][0]["visible"][i*7 + 6] = True  # Show min dropoff-pickup diff
    steps.append(step)

# Create slider
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Name: "},
    pad={"t": 50},
    steps=steps
)]

# Update layout to include a fixed y-axis range (0 to 50)
fig.update_layout(
    sliders=sliders,
    title="Bike Metrics (Average, Max, Min, Max Capacity) & Dropoff-Pickup Diff Over Time",
    xaxis_title="Month",
    yaxis_title="Values",
    yaxis_range=[0, 50],  # Set y-axis range from 0 to 50
    legend_title="Metrics"
)

# Show plot
fig.show()


In [ ]:
# of unique dropoff_hub_id	- # of unique pickup_hub_id -> Average, max, min